In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [5]:
df = pd.read_csv('train.csv', usecols= ['Age', 'Fare', 'Survived'])

In [6]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [23]:
df.dropna(inplace=True)

In [24]:
X= df.iloc[:,1:]
y=df.iloc[:,0]

In [25]:
X.head()

,Age,Fare
0,22.0,7.2500
1,38.0,71.2833
2,26.0,7.9250
3,35.0,53.1000
4,35.0,8.0500


In [26]:
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, random_state=42)

In [27]:
X_train.head(2)

,Age,Fare
328,31.0,20.5250
73,26.0,14.4542


In [28]:
y_train.head(2)

328    1
73     0
Name: Survived, dtype: int64

In [29]:
clf= DecisionTreeClassifier()

In [30]:
clf.fit(X_train, y_train)
y_pred= clf.predict(X_test)

In [31]:
accuracy_score(y_test, y_pred)

0.6363636363636364

In [32]:
np.mean(cross_val_score(clf, X,y, cv=10, scoring='accuracy'))

0.6373043818466354

In [33]:
kbin_age= KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
kbin_fare= KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')

In [34]:
trf= ColumnTransformer ([
    ('first', kbin_age, [0]),
    ('second', kbin_age, [1])
])

In [36]:
X_train_trf= trf.fit_transform(X_train)
X_test_trf= trf.transform(X_test)

In [43]:
trf.named_transformers_['first'].bin_edges_

array([array([ 0.42, 14.  , 19.  , 22.  , 25.  , 28.5 , 32.  , 36.  , 42.  ,
              50.  , 80.  ])                                                ],
      dtype=object)

In [44]:
trf.named_transformers_['second'].bin_edges_

array([array([  0.    ,   7.75  ,   7.8958,   9.225 ,  13.    ,  15.75  ,
               26.    ,  29.125 ,  51.4792,  82.1708, 512.3292])         ],
      dtype=object)

In [45]:
output = pd.DataFrame({
    'age':X_train_trf[:,0],
    'age_trf':X_train_trf[:,0],
    'fare':X_train['Fare'],
    'fare_trf':X_train_trf[:,1]
})

In [46]:
output['age_labels'] = pd.cut(x=X_train['Age'],
                                    bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output['fare_labels'] = pd.cut(x=X_train['Fare'],
                                    bins=trf.named_transformers_['second'].bin_edges_[0].tolist())



In [47]:
output.sample(10)

,age,age_trf,fare,fare_trf,age_labels,fare_labels
873,8.0,8.0,9.0000,2.0,"(42.0, 50.0]","(7.896, 9.225]"
157,5.0,5.0,8.0500,2.0,"(28.5, 32.0]","(7.896, 9.225]"
371,1.0,1.0,6.4958,0.0,"(14.0, 19.0]","(0.0, 7.75]"
52,8.0,8.0,76.7292,8.0,"(42.0, 50.0]","(51.479, 82.171]"
812,6.0,6.0,10.5000,3.0,"(32.0, 36.0]","(9.225, 13.0]"
874,4.0,4.0,24.0000,5.0,"(25.0, 28.5]","(15.75, 26.0]"
400,7.0,7.0,7.9250,2.0,"(36.0, 42.0]","(7.896, 9.225]"
641,3.0,3.0,69.3000,8.0,"(22.0, 25.0]","(51.479, 82.171]"
322,5.0,5.0,12.3500,3.0,"(28.5, 32.0]","(9.225, 13.0]"
281,4.0,4.0,7.8542,1.0,"(25.0, 28.5]","(7.75, 7.896]"


In [49]:
clf = DecisionTreeClassifier()

clf.fit(X_train_trf, y_train)
y_pred2= clf.predict(X_test_trf)

In [50]:
accuracy_score(y_test,y_pred2)

0.6223776223776224

In [51]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(clf, X,y, cv=10, scoring='accuracy'))

0.6303403755868544

In [52]:
def discretize(bins, strategy):
    kbin_age= KBinsDiscretizer(n_bins=10, encode='ordinal', strategy=strategy)
    kbin_fare= KBinsDiscretizer(n_bins=10, encode='ordinal', strategy=strategy)

    trf= ColumnTransformer ([
    ('first', kbin_age, [0]),
    ('second', kbin_age, [1])
    ])

    X_trf = trf.fit_transform(X)
    np.mean(cross_val_score(clf, X,y, cv=10, scoring='accuracy'))
    
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    plt.hist(X['Age'])
    plt.title("Before")

    plt.figure(figsize=(14,4))
    plt.subplot(122)
    plt.hist(X_trf[:,0], color='red')
    plt.title("After")
    
    plt.show()

    plt.figure(figsize=(14,4))
    plt.subplot(121)
    plt.hist(X['Fare'])
    plt.title("Before")

    plt.figure(figsize=(14,4))
    plt.subplot(122)
    plt.hist(X_trf[:,1], color='red')
    plt.title("After")
    
    plt.show()
